In [ ]:
import numpy as np
import pathlib
import pyexr
import rawpy
import tensorflow as tf

# Prevent TensorFlow from allocating all GPU memory.
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
from cnn_demosaic import exposure
from cnn_demosaic import exposure_model
from matplotlib import pyplot as plt

In [ ]:
def display_image(image):
    plt.imshow(image, vmin=0, vmax=1, cmap="gray")
    plt.show()


def to_color_arr(img_arr):
    """Flatten the 2D image to an array."""
    rgb_arr = img_arr[:, :, :3]
    return rgb_arr.reshape((rgb_arr.size // 3, 3))

In [ ]:
EXPOSURE_MODEL_PATH = "./exp_model_32_16_4.weights.h5"
EXPOSURE_MODEL_PATH_NEW = "./exp_model_32_16_5.weights.h5"

DATASET_PREFIX = "/media/jake/Media/datasets/fuji_raw/xe2"

TEST_EXR_PATH = f"{DATASET_PREFIX}/train/DSCF5796.exr"

TRAINING_PATH = pathlib.Path(f"{DATASET_PREFIX}/train")
VALIDATION_PATH = pathlib.Path(f"{DATASET_PREFIX}/val")

VALIDATION_FILES = list(VALIDATION_PATH.glob("*.exr"))

In [ ]:
exp_model = exposure_model.create_exposure_model(EXPOSURE_MODEL_PATH)
exp = exposure.Exposure(exp_model)

exp_model_new = exposure_model.create_exposure_model(EXPOSURE_MODEL_PATH_NEW)
exp_new = exposure.Exposure(exp_model_new)

In [ ]:
def process_exposure(exr_path, processor):
    exr_img_arr = pyexr.read(f"{exr_path}")[:, :, :3]
    orig_shape = exr_img_arr.shape
    output_arr = processor.process(to_color_arr(exr_img_arr))
    display_image(np.asarray(output_arr).reshape(orig_shape))

In [ ]:
process_exposure(TEST_EXR_PATH, exp)
process_exposure(TEST_EXR_PATH, exp_new)

In [ ]:
for f in VALIDATION_FILES:
    process_exposure(f, exp)
    process_exposure(f, exp_new)